## Fixed Income Factor Models Review

Models under consideration:

1. Momentum Factor 
2. Value Factor
3. Low Volatility Factor
4. VIX based Fixed Income Dynamic Allocation Model

*All models are rebalanced on month end

In [252]:
#nbi:hide_in
import warnings
warnings.filterwarnings("ignore")

In [360]:
#nbi:hide_in
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import edhec_risk_kit as erk
import edhec_risk_kit_206 as erk1
import yfinance as yf
import statsmodels.api as sm
import seaborn as sns
from datetime import date
from pandas_datareader import data
import pandas_datareader.data as web
import quantstats as qs
import fixed_income_factors as fif
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import plotly.express as px
import plotly.graph_objects as go
import plotly


%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#nbi:hide_in
import quandl
quandl.ApiConfig.api_key = 'KZ69tzkHfXscfQ1qcJ5K'

## Raw Data Gathering
1. Bond Total Return Indices
2. Effective Yields of Bond Indices
3. Inflation Expectations


#### Universe: 

| Sr. No. | BBG Tickers | Category                   | ETF       | Index                                                                            | Short Name           |
|---------|-------------|----------------------------|-----------|----------------------------------------------------------------------------------|----------------------|
| 1       | LD12TRUU    | Sovereign                  | BIL       | Bloomberg Barclays US Treasury Bill 1-3Month                                     | US 1-3M T-Bills      |
| 2       | LT01TRUU    | Sovereign                  | SPTS      | Bloomberg Barclays 1-3 Year U.S. Treasury Index                                  | US 1-3Y Treasuries   |
| 3       | LT08TRUU    | Sovereign                  | SPTI      | Bloomberg Barclays Intermediate Term U.S. Treasury Index                         | US Int Treasuries    |
| 4       | LT09TRUU    | Sovereign                  | IEF       | Bloomberg Barclays 7-10 Year U.S. Treasury Index                                 | US 7-10Y Treasuries  |
| 5       | LUTLTRUU    | Sovereign                  | SPTL      | Bloomberg Barclays Long U.S. Treasury Index                                      | US Long Treasuries   |
| 6       | SPBDXUTR    | Sovereign                  | IGOV      | S&P International Sovereign Ex-U.S. Bond Index                                   | Intl Sovereign Bonds |
| 7       | BTSYTRUU    | Sovereign                  | BWX       | Bloomberg Barclays Global Treasury Total Return Index                            | Global Treasuries    |
| 8       | BCIT1T      | Sovereign TIPS             | SPIP      | Bloomberg Barclays US Government Inflation-linked Bond Index                     | US TIPS              |
| 9       | LUMSTRUU    | MBS                        | SPMB      | Bloomberg Barclays U.S. MBS Index                                                | US MBS               |
| 10      | BUC1TRUU    | Investment Grade Corporate | VCSH      | Bloomberg Barclays U.S. 1-5 Year Corporate Index                                 | US IG 1-5Y           |
| 11      | LF99TRUU    | Investment Grade Corporate | SPSB      | Bloomberg Barclays U.S. 1-3 Year Corporate Bond Index                            | US IG 1-3Y           |
| 12      | LD06TRUU    | Investment Grade Corporate | SPIB      | Bloomberg Barclays Intermediate US Corporate Index                               | US IG Int            |
| 13      | LD07TRUU    | Investment Grade Corporate | SPLB/VCLT | Bloomberg Barclays Long U.S. Corporate Index                                     | US IG Long           |
| 14      | LG38TRUU    | Investment Grade Corporate | IAGG      | Bloomberg Barclays Global Aggregate ex USD 10% Issuer Capped (Hedged) Index      | Global IG            |
| 15      | LUICTRUU    | Investment Grade Corporate | IGIB      | Bloomberg Barclays U.S. Intermediate Credit Index                                | Developed Mkts IG    |
| 16      | LEGATRUU    | Global Aggregate           | AGGG LN   | Bloomberg Barclays Global Aggregate Bond Index                                   | Global Aggregate     |
| 17      | LECPTREU    | European IG Corporates     | EUCO LN   | Bloomberg Barclays Euro Aggregate Corporate Total Return Index Value Unhedged EU | Euro Aggregate       |
| 18      | LF98TRUU    | High Yield Corporate       | BKHY      | Bloomberg Barclays US Corporate High Yield Index                                 | US HY Beta           |
| 19      | LHVLTRUU    | US High Yield              | JNK       | Bloomberg Barclays High Yield Very Liquid Index                                  | US HY                |
| 20      | FRED Data   | EM Crossover               | NA        | ICE BofA Crossover Emerging Markets Corporate Plus Index Total Return            | US Crossover         |
| 21      | BEBGTRUU    | EM High Yield              | EMHY/HYEM | Bloomberg Barclays Emerging Markets High Yield Total Return Index                | EM HY                |
| 22      | LP01TREU    | European High Yield        | SYBJ/JNKE | Bloomberg Barclays Pan-European High Yield Total Return Index                    | Euro HY              |

In [56]:
#nbi:hide_in
indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
indices.dropna(inplace=True)

> Real Yields (Expected Inflation) are calculated by reducing the inflation expectations from the index yields.


> Real Yields (Actual Inflation) are calculated by reducing the average CPI from the index yields based on the Weighted Average Maturity (WAM) of the indices (for example: 5Y monthly average annual inflation for indices with 5Y WAM)


Data Source: [Cleveland Fed Inflation Expectations](https://www.clevelandfed.org/our-research/indicators-and-data/inflation-expectations.aspx)

In [52]:
#nbi:hide_in
real_yields = fif.import_yield_data(yields_d='Expected_Infl')

nom_yields = fif.import_yield_data(yields_d='Nominal')

act_yields = fif.import_yield_data(yields_d='Actual_Infl')

### Defining the Function for Fixed Income Factors Backtest

1. Requires Real Yields of All Securities & Total Return Indices of the same as Raw Data
2. Input Parameters
   
   > Number of Top Scoring Securities to go buy (Max Top 3, weighted equally)
   
   > Sample Period Start Date & End Date
   
   > "Yes" for Long/Short and "No" for Long Only Backtest Results for input variable 'Long/Short'
   
3. Specific to Momentum Factor
   
   > Lookback Period for Momentum in Total Return Bond Indices (in Days) 

4. Specific to Value Factor
   
   > Lookback Period for Real Yields Z-Score Calculation (in Years)
   
   > Lookback Period for Smoothening Z-Score (in Days) to avoid timing luck factor
  
5. Specific to Low Volatility Factor
    
    > Lookback Period for Yield Volatility Calculation

6. VIX High Yield and Duration Allocation Model:

    > Lookback Period for calculating today's VIX as a percentage of the lookback period's maximum VIX value on a rolling basis
    
    > Smoothening Period i.e. a moving average of the VIX percentile for avoiding luck factor

## Generating Portfolio Tearsheet to compare with a benchmark i.e. Equally Weighted of all the above securities

In [254]:
#nbi:hide_in
import fixed_income_factors as fif
#Inputs

zscore_lookback = 3       #Number of Years
z_score_smoothening=7   #Number of Days
n_securities=3            #Number of Securities for Long only or Long/Short Portfolio
rebal = 'Quarterly'
lev_factor=1
spread=0
trans_cost = 0
yield_type='Real Yields - Expected Inflation'
long_short='No'          #Long/Short or Long/Only as Output
duration_adj = 'Yes'
sample_start='2003'       #Start of Sample Period
sample_end='2015'         #End of Sample Period

#Momentum
lookback_period = 21     #Quarterly Returns as lookback 

## Momentum Factor Summary Statistics

> Goes long the top securities with highest total return based on the lookback period selected. 

### Interactive Momentum Factor Backtest

In [64]:
#nbi:hide_in
cost = quandl.get("FRED/DFF", start_date="1995-12-31")/(252*100)
cost.index.name = 'Date'
cost.columns = ['EFFR']

In [337]:
#nbi:hide_in
indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
indices.dropna(inplace=True)
momentum_factor  = fif.momentum_factor_backtest(indices, rebal, lookback_period, duration_adj, n_securities, long_short, sample_start, sample_end,  cost, spread, lev_factor, trans_cost)

In [338]:
#nbi:hide_in
def momentum_interactive(lookback_period, rebal, duration_adj, n_securities, long_short, lev_factor, spread, trans_cost, sample_start, sample_end):
    indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
    indices.dropna(inplace=True)
    momentum_factor  = fif.momentum_factor_backtest(indices, rebal, lookback_period, duration_adj, n_securities, long_short, sample_start, sample_end, cost, spread, lev_factor, trans_cost)
    fif.plot_chart(momentum_factor[['Momentum Factor', 'EW', 'US HY', 'Global Treasuries', 'US 7-10Y Treasuries', 'Global Aggregate', 'US IG Long']], sample_start=sample_start, sample_end=sample_end, long_short=long_short, factor_name = 'Momentum')
    return erk.summary_stats1(momentum_factor,0,252).sort_values('Total Return').T

def momentum_interact():
    style = {'description_width': 'initial'}    
    return interact(momentum_interactive,lookback_period = widgets.IntSlider(21, 21, 252, 1, style=style, description='Lookback Period (Days): '),
                         duration_adj = widgets.Dropdown(options = ['Yes', 'No'], value = 'Yes',style=style, description='Duration Adjustment: '), 
                         n_securities = widgets.IntSlider(3,1,3,1, style=style, description='No. of Securities: '),
                         lev_factor = widgets.BoundedFloatText(value=1,min=1,max=10,step=0.1, style=style, description='Leverage Factor: '),
                         spread = widgets.BoundedFloatText(value=0,min=0,max=1000,step=1, style=style, description='Spread over Fed Funds Rate (in bps): '),
                         trans_cost = widgets.BoundedFloatText(value=0,min=0,max=500,step=1, style=style, description='Transaction Cost (in bps): '),
                         rebal = widgets.Dropdown(options = ['Monthly', 'Quarterly'], value = 'Quarterly',style=style, description='Rebalancing Frequency: '),
                         long_short= widgets.Dropdown(options = ['Yes', 'No'], value = 'No',style=style, description='Long/Short: '), 
                         sample_start = widgets.DatePicker(value=momentum_factor.index[0],style=style, description = 'Sample Start: '),
                         sample_end = widgets.DatePicker(value=pd.to_datetime('31-Dec-2015'),style=style,  description = 'Sample End: '))


momentum_interact();

interactive(children=(IntSlider(value=21, description='Lookback Period (Days): ', max=252, min=21, style=Slide…

In [28]:
#nbi:hide_in
#qs.reports.html(momentum_factor['Momentum Strategy'], momentum_factor['EW'], output='Momentum.html', title='Momentum Factor Tearsheet')
#import webbrowser
#new = 2
#url = "Momentum.html"
#webbrowser.open(url,new=new)

## Value Factor Summary Statistics

> Go long the top securities which have the highest Z-Score of Real Yields when compared to the past X periods on a rolling basis.

### Interactive Value Factor Backtest

In [342]:
#nbi:hide_in
indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
indices.dropna(inplace=True)
value_factor  = fif.value_factor_backtest(nom_yields, act_yields, real_yields, yield_type, rebal, indices, zscore_lookback, z_score_smoothening, n_securities, long_short, sample_start, sample_end, cost, spread, lev_factor, trans_cost)

In [344]:
#nbi:hide_in
def value_interactive_stats(rebal, yield_type, zscore_lookback, z_score_smoothening, n_securities, long_short,lev_factor, spread, trans_cost, sample_start, sample_end):
    indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
    indices.dropna(inplace=True)
    value_factor  = fif.value_factor_backtest(nom_yields, act_yields, real_yields, yield_type, rebal, indices, zscore_lookback, z_score_smoothening, n_securities, long_short, sample_start, sample_end, cost, spread, lev_factor, trans_cost)
    fif.plot_chart(value_factor[['Value Factor', 'EW', 'US HY', 'EM HY', 'US 7-10Y Treasuries', 'Global Aggregate', 'US IG Long']], sample_start='2008', sample_end='2015', long_short=long_short, factor_name = 'Value')
    return erk.summary_stats1(value_factor,0,252).sort_values('Total Return').T


    
style = {'description_width': 'initial'}
layout = {'width': '400px'}
interact(value_interactive_stats,zscore_lookback = widgets.IntSlider(3, 1, 10, 1, style=style, layout=layout, description='Z-Score Lookback Period (Years)'),
                         z_score_smoothening =  widgets.IntSlider(7, 1, 700, 50, style=style, layout=layout, description='Z-Score Smoothening Period (Days)'), 
                         rebal = widgets.Dropdown(options = ['Monthly', 'Quarterly'], value = 'Quarterly',style=style, description='Rebalancing Frequency'),
                         lev_factor = widgets.BoundedFloatText(value=1,min=1,max=10,step=0.1, style=style, description='Leverage Factor: '),
                         spread = widgets.BoundedFloatText(value=0,min=0,max=1000,step=1, style=style, description='Spread over Fed Funds Rate (in bps): '),
                         trans_cost = widgets.BoundedFloatText(value=0,min=0,max=500,step=1, style=style, description='Transaction Cost (in bps): '),
                         yield_type = widgets.Dropdown(options = ['Real Yields - Expected Inflation','Real Yields - Actual Inflation', 'Nominal Yields'], value = 'Real Yields - Expected Inflation',style=style, description='Yield Type'),
                         n_securities = widgets.IntSlider(3,1,3,1, style=style, description='No. of Securities'),
                         long_short= widgets.Dropdown(options = ['Yes', 'No'], value = 'No',style=style, description='Long/Short'), 
                         sample_start = widgets.DatePicker(value=value_factor.index[0],style=style, description = 'Sample Start'),
                         sample_end = widgets.DatePicker(value=pd.to_datetime('31-Dec-2015'),style=style,  description = 'Sample End'));

interactive(children=(Dropdown(description='Rebalancing Frequency', index=1, options=('Monthly', 'Quarterly'),…

In [26]:
#nbi:hide_in
#qs.reports.html(value_factor['Value Strategy'], value_factor['EW'], output='Value.html', title='Value Factor Tearsheet')
#import webbrowser
#new = 2
#url = "Value.html"
#webbrowser.open(url,new=new)

## Low Volatility Factor Backtest

> Goes long the top securities with lowest yield volatility (adjusted for duration - divide yields by their WAM) based on the lookback period selected. 

In [345]:
#nbi:hide_in
indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
indices.dropna(inplace=True)
volatility_factor  = fif.volatility_factor_backtest(nom_yields, act_yields, real_yields, yield_type, rebal, indices, lookback_period, n_securities, long_short, sample_start, sample_end, cost, spread, lev_factor, trans_cost)

In [346]:
#volatility_factor.head()

In [347]:
#nbi:hide_in
def vol_interactive(yield_type, rebal, lookback_period, n_securities, long_short, lev_factor, spread, trans_cost, sample_start, sample_end):
    indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
    indices.dropna(inplace=True)
    volatility_factor  = fif.volatility_factor_backtest(nom_yields, act_yields, real_yields, yield_type, rebal, indices, lookback_period, n_securities, long_short, sample_start, sample_end, cost, spread, lev_factor, trans_cost)
    fif.plot_chart(volatility_factor[['Low Volatility Factor', 'EW', 'US HY', 'EM HY', 'US 7-10Y Treasuries', 'Global Aggregate', 'US IG Long']], sample_start='2003', sample_end='2015', long_short='No', factor_name = 'Low Volatility')
    return erk.summary_stats1(volatility_factor,0,252).sort_values('Total Return').T

def vol_interact():
    style = {'description_width': 'initial'}    
    return interact(vol_interactive,lookback_period = widgets.IntSlider(30, 21, 252, 1, style=style, description='Lookback Period'), 
                         n_securities = widgets.IntSlider(3,1,3,1, style=style, description='No. of Securities'),
                         rebal = widgets.Dropdown(options = ['Monthly', 'Quarterly'], value = 'Quarterly',style=style, description='Rebalancing Frequency'),
                         yield_type = widgets.Dropdown(options = ['Real Yields - Expected Inflation','Real Yields - Actual Inflation', 'Nominal Yields'], value = 'Real Yields - Expected Inflation',style=style, description='Yield Type'),
                         long_short= widgets.Dropdown(options = ['Yes', 'No'], value = 'No',style=style, description='Long/Short'), 
                         lev_factor = widgets.BoundedFloatText(value=1,min=1,max=10,step=0.1, style=style, description='Leverage Factor: '),
                         spread = widgets.BoundedFloatText(value=0,min=0,max=1000,step=1, style=style, description='Spread over Fed Funds Rate (in bps): '),
                         trans_cost = widgets.BoundedFloatText(value=0,min=0,max=500,step=1, style=style, description='Transaction Cost (in bps): '),
                         sample_start = widgets.DatePicker(value=volatility_factor.index[0],style=style, description = 'Sample Start'),
                         sample_end = widgets.DatePicker(value=pd.to_datetime('31-Dec-2015'),style=style,  description = 'Sample End'))


vol_interact();

interactive(children=(Dropdown(description='Yield Type', options=('Real Yields - Expected Inflation', 'Real Yi…

In [63]:
#nbi:hide_in
#qs.reports.html(volatility_factor['Low Volatility Strategy'], volatility_factor['EW'], output='Volatility.html', title='Low Volatility Factor Tearsheet')
#import webbrowser
#new = 2
#url = "Volatility.html"
#webbrowser.open(url,new=new)

## EW Composite

In [348]:
#nbi:hide_in
factors = pd.DataFrame(value_factor['Value Factor']).merge(volatility_factor['Low Volatility Factor'], on='Date').merge(pd.DataFrame(momentum_factor), on='Date')
factors['EW Composite'] = factors[['Value Factor', 'Momentum Factor', 'Low Volatility Factor']].mean(axis=1)

## Factor Correlations

In [349]:
#nbi:hide_in
factors[['Value Factor', 'Momentum Factor', 'Low Volatility Factor', ]].corr().round(4)*100

,Value Factor,Momentum Factor,Low Volatility Factor
Value Factor,100.00,36.73,44.96
Momentum Factor,36.73,100.00,37.24
Low Volatility Factor,44.96,37.24,100.00


### Equally Weighted Factor (Long-Only) Return Comparison

In [351]:
#nbi:hide_in
fif.plot_chart(factors[[ 'EW Composite','EW','Momentum Factor', 'Value Factor', 'Low Volatility Factor']], sample_start='2006', sample_end='2015', long_short='No', factor_name = 'EW Composite - Momentum, Value & Low Volatility');

In [352]:
#nbi:hide_in
erk.summary_stats1(factors,0,252).sort_values('Total Return').T

,US 1-3M T-Bills,Euro Aggregate,US 1-3Y Treasuries,Global IG,Intl Sovereign Bonds,Global Treasuries,Global Aggregate,US IG 1-3Y,US Int Treasuries,US IG 1-5Y,...,US 7-10Y Treasuries,US HY,US HY Beta,US IG Long,EM HY,US Long Treasuries,Low Volatility Factor,Momentum Factor,EW Composite,Value Factor
Total Return,4.55,19.25,19.79,24.06,25.94,30.99,33.89,34.42,37.35,41.90,...,66.59,69.20,70.38,77.10,84.11,89.46,108.50,119.83,121.83,133.72
Annualized Return,0.55,2.30,2.24,2.80,3.00,3.46,3.72,3.69,3.97,4.38,...,6.46,6.62,6.72,7.26,7.79,8.16,9.43,10.10,10.28,11.07
Annualized Vol,0.16,15.65,1.46,8.38,9.31,6.86,5.61,1.96,3.51,2.76,...,7.42,7.45,5.93,11.49,6.91,13.57,6.51,6.20,4.77,5.79
Skewness,0.55,0.94,-0.12,0.29,0.30,0.23,0.13,-1.35,0.05,-0.53,...,0.10,-0.84,-1.33,-0.16,-1.59,-0.02,-0.45,0.10,-0.15,-0.18
Kurtosis,65.15,71.22,16.60,9.23,8.77,8.56,7.81,27.89,7.61,12.37,...,5.70,18.49,25.79,4.31,25.71,4.05,6.90,7.75,5.93,7.91
Cornish-Fisher VaR (5%),0.00,-0.03,0.12,0.75,0.83,0.62,0.52,0.17,0.32,0.26,...,0.70,0.70,0.55,1.17,0.66,1.36,0.66,0.55,0.45,0.54
Historic CVaR (5%),0.01,2.03,0.20,1.19,1.31,0.97,0.79,0.29,0.49,0.41,...,1.01,1.21,0.96,1.61,1.11,1.83,0.95,0.91,0.65,0.80
Sharpe Ratio,3.51,0.15,1.54,0.33,0.32,0.51,0.66,1.88,1.13,1.59,...,0.87,0.89,1.13,0.63,1.13,0.60,1.45,1.63,2.16,1.91
Sortino Ratio,2.08,0.19,1.97,0.49,0.48,0.74,0.97,1.96,1.65,1.95,...,1.34,0.95,1.15,0.94,1.15,0.93,1.93,2.17,3.12,2.61
Max Drawdown,-0.17,-26.04,-2.17,-15.02,-15.51,-11.43,-10.82,-7.36,-4.48,-9.66,...,-10.22,-38.56,-35.22,-24.84,-35.91,-18.40,-20.74,-12.06,-14.40,-18.65


In [353]:
#nbi:hide_in
spy = yf.download('SPY', progress=False)['Adj Close'];
spy=pd.DataFrame(spy);
spy.columns = ['SPY'];

In [354]:
#nbi:hide_in
#qs.reports.html(factors['EW Composite']['2009-05':], spy.pct_change().dropna() , output='Composite.html', title='EW Composite (Momentum, Value, Volatility) Factor Tearsheet')
#import webbrowser
#new = 2
#url = "Composite.html"
#webbrowser.open(url,new=new)

## VIX Switching Strategy

>  Uses VIX level as a percentile of the past lookback period to decide allocation between US Treasuries (Intermediate term, 7-10Y, 20Y+)  and High Yield Bonds (US High Yield, EM High Yield, Euro High Yield, Crossover i.e. BBB-BB).

> For example, if on rebalancing date, current VIX is at 40th percentile of its past 'X' years range, then the model invests 40% of portfolio in US Treasuries (equally weighted in the above mentioned 3 securities/indices) and 60% in High Yield (equally weighted in the above mentioned 4 securities/indices).

> Under 'Compare With' dropdown, Equally Weighted is an equally weighted portfolio of all the above 7 securities/indices.

> Lookback Period is the period to be used (in years) to calculate today's VIX as a percentage of the past 'X' years max VIX.

> Since VIX itself is volatile, VIX Smoothening Period can be used to remove any luck factor. For example a VIX smoothening period of 7 days will mean that after calculating the VIX percentile everyday on a rolling basis, a 7-period moving average is calculated and that value is used for allocation on the rebalancing date.

In [355]:
#Data Collection and Manipulation
#nbi:hide_in
indices = pd.read_excel('Total Return Index - FI.xlsx', header=0, index_col=0, parse_dates=True)
indices.dropna(inplace=True)
col = indices.columns
duration = indices[['US Int Treasuries', 'US 7-10Y Treasuries', 'US Long Treasuries', 'Intl Sovereign Bonds', 'Global Treasuries']]
high_yield = indices[['US HY', 'US HY Beta','US Crossover', 'EM HY', 'Euro HY']]

In [356]:
#nbi:hide_in
#qs.reports.html(vix_model['Portfolio'], vix_model.drop(['Portfolio'], axis=1).mean(axis=1), output='VIX_Model.html', title='VIX Allocation Model Tearsheet')
#import webbrowser
#new = 2
#url = "VIX_Model.html"
#webbrowser.open(url,new=new)

### Interactive Paramter Sensitivity Testing for VIX Based Fixed Income Allocation Model 

In [357]:
#nbi:hide_in
import ipywidgets as widgets
from IPython.display import display

In [361]:
#nbi:hide_in
vix = pd.DataFrame(yf.download('^VIX', start='1999-01-01', progress=False)['Adj Close'])

def show_vix_model(lookback_period, vix_smooth_period, sample_start, sample_end):
    vix_model = fif.fi_vix_allocator(duration, high_yield, vix, lookback_period, vix_smooth_period, sample_start, sample_end)
    vix_model['EW'] = vix_model.drop('Portfolio', axis=1).mean(axis=1)
    #nbi:hide_in
    fif.plot_chart(vix_model[['Portfolio','EW', 'US HY', 'EM HY', 'US 7-10Y Treasuries', 'US Long Treasuries']], sample_start='2006', sample_end='2015', long_short='No', vix='No');
    #b = (1+vix_model[['Portfolio','EW', 'US HY', 'EM HY', 'US 7-10Y Treasuries', 'US Long Treasuries']]).cumprod().plot(figsize=(13,8), fontsize=13)
    #b.annotate("Annualized Return(CAGR): "+str((erk.annualized_ret(vix_model['Portfolio'], 252)*100).round(2)) +"%", xy=(.05, .90), xycoords='axes fraction', fontsize=14)
    #b.annotate("Sharpe Ratio: "+str(erk.sharpe_ratio(vix_model['Portfolio'], 252, 0.0).round(2)), xy=(.05, .85), xycoords='axes fraction', fontsize=14)
    #plt.title('VIX Allocation Strategy In-Sample Backtest', fontsize=14)
    #plt.legend(loc=(1,0.1))
    return erk.summary_stats1(vix_model,0,252).sort_values('Total Return').T
    

In [362]:
#nbi:hide_in

style = {'description_width': 'initial'}    
interact(show_vix_model,lookback_period = widgets.IntSlider(min=1, max=10, step=1, value=5, description="Lookback Period (Yrs): ",style = {'description_width': '200px'}, layout = {'width': '400px'}),
                                   vix_smooth_period = widgets.IntSlider(min=1, max=15, step=1, value=7, description="Smoothening Period (Days): ",style = {'description_width': '200px'}, layout = {'width': '400px'}),
                                   #rebal = widgets.Dropdown(options = ['Monthly', 'Quarterly'], value = 'Monthly',style=style, description='Rebalancing Frequency'),
                                   #compare_with = widgets.Dropdown(options = [('All',['EW', 'US HY', 'EM HY', 'US 7-10Y Treasuries','US Long Treasuries']), ('Equally Weighted','EW'), ('US High Yield','US HY'),
                                                                              #('EM High Yield', 'EM HY'), ('US Treasuries 7-10Y', 'US 7-10Y Treasuries')], value =['EW', 'US HY', 'EM HY', 'US 7-10Y Treasuries','US Long Treasuries'] ,style=style, description='Compare With: '),
                                   sample_start = widgets.DatePicker(value=pd.to_datetime('27-Feb-2004'),style=style, description = 'Sample Start'),
                                   sample_end = widgets.DatePicker(value=pd.to_datetime('31-Dec-2015'),style=style,  description = 'Sample End'),
                                   );

interactive(children=(IntSlider(value=5, description='Lookback Period (Yrs): ', layout=Layout(width='400px'), …